# UDF vs Higher order functions

In [1]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("UDF vs Higher Order Functions") \
    .master("local[*]") \
    .getOrCreate()

spark

In [47]:
# Example Data Frame

_data = [
    [1, ["Bangalore", "Mumbai", "Pune", "Indore"]],
    [2, ["Bangalore"]],
    [3, []],
    [4, ["Kolkata", "Bhubaneshwar"]],
    [5, ["Bangalore", "Mumbai", "Pune", "Indore", "Ahmedabad", "Suratkal"]],
    [6, ["Delhi", "Mumbai", "Kolkāta", "Bangalore", "Chennai", "Hyderābād", "Pune", "Ahmedabad", "Sūrat", "Lucknow", "Jaipur", "Cawnpore", "Mirzāpur", "Nāgpur", "Ghāziābād", "Indore", "Vadodara", "Vishākhapatnam", "Bhopāl", "Chinchvad", "Patna", "Ludhiāna", "Āgra", "Kalyān", "Madurai", "Jamshedpur", "Nāsik", "Farīdābād", "Aurangābād", "Rājkot", "Meerut", "Jabalpur", "Thāne", "Dhanbād", "Allahābād", "Vārānasi", "Srīnagar", "Amritsar", "Alīgarh", "Bhiwandi", "Gwalior", "Bhilai", "Hāora", "Rānchi", "Bezwāda", "Chandīgarh", "Mysore", "Raipur", "Kota", "Bareilly", "Jodhpur", "Coimbatore", "Dispur", "Guwāhāti", "Solāpur", "Trichinopoly", "Hubli", "Jalandhar", "Bhubaneshwar", "Bhayandar", "Morādābād", "Kolhāpur", "Thiruvananthapuram", "Sahāranpur", "Warangal", "Salem", "Mālegaon", "Kochi", "Gorakhpur", "Shimoga", "Tiruppūr", "Guntūr", "Raurkela", "Mangalore", "Nānded", "Cuttack", "Chānda", "Dehra Dūn", "Durgāpur", "Āsansol", "Bhāvnagar", "Amrāvati", "Nellore", "Ajmer", "Tinnevelly", "Bīkaner", "Agartala", "Ujjain", "Jhānsi", "Ulhāsnagar", "Davangere", "Jammu", "Belgaum", "Gulbarga", "Jāmnagar", "Dhūlia", "Gaya", "Jalgaon", "Kurnool", "Udaipur", "Bellary", "Sāngli", "Tuticorin", "Calicut", "Akola", "Bhāgalpur", "Sīkar", "Tumkūr", "Quilon", "Muzaffarnagar", "Bhīlwāra", "Nizāmābād", "Bhātpāra", "Kākināda", "Parbhani", "Pānihāti", "Lātūr", "Rohtak", "Rājapālaiyam", "Ahmadnagar", "Cuddapah", "Rājahmundry", "Alwar", "Muzaffarpur", "Bilāspur", "Mathura", "Kāmārhāti", "Patiāla", "Saugor", "Bijāpur", "Brahmapur", "Shāhjānpur", "Trichūr", "Barddhamān", "Kulti", "Sambalpur", "Purnea", "Hisar", "Fīrozābād", "Bīdar", "Rāmpur", "Shiliguri", "Bāli", "Pānīpat", "Karīmnagar", "Bhuj", "Ichalkaranji", "Tirupati", "Hospet", "Āīzawl", "Sannai", "Bārāsat", "Ratlām", "Handwāra", "Drug", "Imphāl", "Anantapur", "Etāwah", "Rāichūr", "Ongole", "Bharatpur", "Begusarai", "Sonīpat", "Rāmgundam", "Hāpur", "Uluberiya", "Porbandar", "Pāli", "Vizianagaram", "Puducherry", "Karnāl", "Nāgercoil", "Tanjore", "Sambhal", "Naihāti", "Secunderābād", "Kharagpur", "Dindigul", "Shimla", "Ingrāj Bāzār", "Ellore", "Puri", "Haldia", "Nandyāl", "Bulandshahr", "Chakradharpur", "Bhiwāni", "Gurgaon", "Burhānpur", "Khammam", "Madhyamgram", "Ghāndīnagar", "Baharampur", "Mahbūbnagar", "Mahesāna", "Ādoni", "Rāiganj", "Bhusāval", "Bahraigh", "Shrīrāmpur", "Tonk", "Sirsa", "Jaunpur", "Madanapalle", "Hugli", "Vellore", "Alleppey", "Cuddalore", "Deo", "Chīrāla", "Machilīpatnam", "Medinīpur", "Bāramūla", "Chandannagar", "Fatehpur", "Udipi", "Tenāli", "Sitalpur", "Conjeeveram", "Proddatūr", "Navsāri", "Godhra", "Budaun", "Chittoor", "Harīpur", "Saharsa", "Vidisha", "Pathānkot", "Nalgonda", "Dibrugarh", "Bālurghāt", "Krishnanagar", "Fyzābād", "Silchar", "Shāntipur", "Hindupur"]]
]

_cols = ["id", "cities"]

# Create Data Frame
df = spark.createDataFrame(data = _data, schema = _cols)
df.printSchema()s
df.show()

root
 |-- id: long (nullable = true)
 |-- cities: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---+--------------------+
| id|              cities|
+---+--------------------+
|  1|[Bangalore, Mumba...|
|  2|         [Bangalore]|
|  3|                  []|
|  4|[Kolkata, Bhubane...|
|  5|[Bangalore, Mumba...|
|  6|[Delhi, Mumbai, K...|
+---+--------------------+



In [48]:
# Lets create a simple Python decorator - {get_time} to get the execution timings
# If you dont know about Python decorators - check out : https://www.geeksforgeeks.org/decorators-in-python/
import time

def get_time(func):
    def inner_get_time() -> str:
        start_time = time.time()
        func()
        end_time = time.time()
        return (f"Execution time: {(end_time - start_time)*1000} ms")
    print(inner_get_time())

In [67]:
# UDF for len of cities
from pyspark.sql.functions import udf

def len_of_cities(col):
    _len = 0
    for i in col:
        _len += len(i)
    return _len

len_of_cities_udf = udf(lambda x: len_of_cities(x))

In [80]:
# Getting the execution time using the UDF

@get_time
def x(): df.withColumn("len_of_cities", count_of_cities_udf("cities")).show()

+---+--------------------+-------------+
| id|              cities|len_of_cities|
+---+--------------------+-------------+
|  1|[Bangalore, Mumba...|           25|
|  2|         [Bangalore]|            9|
|  3|                  []|            0|
|  4|[Kolkata, Bhubane...|           19|
|  5|[Bangalore, Mumba...|           42|
|  6|[Delhi, Mumbai, K...|         1806|
+---+--------------------+-------------+

Execution time: 395.79248428344727 ms


In [79]:
# Using Higher Order Function
from pyspark.sql.functions import aggregate, lit, length, size

@get_time
def x(): df.withColumn("len_of_cities", aggregate("cities", lit(0), lambda x, y: x + length(y))).show()

+---+--------------------+-------------+
| id|              cities|len_of_cities|
+---+--------------------+-------------+
|  1|[Bangalore, Mumba...|           25|
|  2|         [Bangalore]|            9|
|  3|                  []|            0|
|  4|[Kolkata, Bhubane...|           19|
|  5|[Bangalore, Mumba...|           42|
|  6|[Delhi, Mumbai, K...|         1806|
+---+--------------------+-------------+

Execution time: 286.47470474243164 ms
